In [4]:
import optuna
import numpy as np
from sklearn.model_selection import KFold,train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import pandas as pd

In [2]:
#数据
data_train=pd.read_csv("/Users/macbookair/Documents/python 2/data.csv")
data_train_price=data_train["价格"]
data_train=data_train.drop(columns=["Unnamed: 0","价格","text"])
X=data_train
y=data_train_price



In [11]:
X_sampled, _, y_sampled, _ = train_test_split(X, y, random_state=42, shuffle=True)


In [12]:
# 总试验次数
N_TRIALS = 50
progress_bar = tqdm(total=N_TRIALS)


120it [1:35:40, 47.84s/it]                               | 0/50 [00:00<?, ?it/s]


In [13]:
# 定义每个模型的调参目标函数
def create_objective(model_name):
    def objective(trial):
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        rmse_scores = []

        for train_idx, val_idx in kf.split(X_sampled):
            X_train, X_val = X_sampled.iloc[train_idx], X_sampled.iloc[val_idx]
            y_train, y_val = y_sampled.iloc[train_idx], y_sampled.iloc[val_idx]

            if model_name == 'xgb':
                params = {
                    'max_depth': trial.suggest_int('max_depth', 3, 10),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                    'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
                }
                model = XGBRegressor(**params, verbosity=0)
            elif model_name == 'lgb':
                params = {
                    'num_leaves': trial.suggest_int('num_leaves', 20, 100),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                    'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
                }
                model = LGBMRegressor(**params)
            elif model_name == 'cat':
                params = {
                    'depth': trial.suggest_int('depth', 3, 10),
                    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
                    'iterations': trial.suggest_int('iterations', 100, 1000)
                }
                model = CatBoostRegressor(**params, verbose=0)
            elif model_name == 'rf':
                params = {
                    'max_depth': trial.suggest_int('max_depth', 5, 20),
                    'n_estimators': trial.suggest_int('n_estimators', 100, 1000)
                }
                model = RandomForestRegressor(**params)
            else:
                raise ValueError("Unknown model")

            model.fit(X_train, y_train)
            preds = model.predict(X_val)
            rmse = mean_squared_error(y_val, preds, squared=False)
            rmse_scores.append(rmse)

        return np.mean(rmse_scores)
    return objective

In [14]:
def callback(study, trial):
    progress_bar.update(1)

In [15]:
# 创建一个函数批量调参
def tune_all_models():
    models = ['xgb', 'lgb', 'cat', 'rf']
    best_params = {}

    for model_name in tqdm(models, desc="Tuning models"):
        study = optuna.create_study(direction='minimize')
        study.optimize(create_objective(model_name), n_trials=30,callbacks=[callback])
        best_params[model_name] = study.best_params

    return best_params


In [16]:
# 执行调参
best_hyperparams = tune_all_models()
best_hyperparams


Tuning models:   0%|                                      | 0/4 [00:00<?, ?it/s][I 2025-06-04 00:51:31,544] A new study created in memory with name: no-name-1f92eaf7-0471-4ad7-a478-76d2e8296cce
[I 2025-06-04 00:51:44,981] Trial 0 finished with value: 751375.3734359893 and parameters: {'max_depth': 6, 'learning_rate': 0.2872039039390133, 'n_estimators': 535}. Best is trial 0 with value: 751375.3734359893.

  2%|▉                                           | 1/50 [00:18<15:12, 18.61s/it][I 2025-06-04 00:52:11,856] Trial 1 finished with value: 750305.1649093869 and parameters: {'max_depth': 8, 'learning_rate': 0.09541100974833464, 'n_estimators': 770}. Best is trial 1 with value: 750305.1649093869.

  4%|█▊                                          | 2/50 [00:45<18:46, 23.47s/it][I 2025-06-04 00:52:18,879] Trial 2 finished with value: 781796.7353751882 and parameters: {'max_depth': 8, 'learning_rate': 0.20868223113430293, 'n_estimators': 143}. Best is trial 1 with value: 750305.1649093869.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008853 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1069
[LightGBM] [Info] Number of data points in the train set: 50479, number of used features: 90
[LightGBM] [Info] Start training from score 1966133.280770
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007811 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1066
[LightGBM] [Info] Number of data points in the train set: 50479, number of used features: 90
[LightGBM] [Info] Start training from score 1959077.578795
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007795 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory 

[I 2025-06-04 01:02:43,913] Trial 0 finished with value: 731489.4631806696 and parameters: {'num_leaves': 39, 'learning_rate': 0.2548768558471221, 'n_estimators': 923}. Best is trial 0 with value: 731489.4631806696.

 62%|██████████████████████████▋                | 31/50 [11:17<06:04, 19.16s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008780 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1069
[LightGBM] [Info] Number of data points in the train set: 50479, number of used features: 90
[LightGBM] [Info] Start training from score 1966133.280770
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008887 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1066
[LightGBM] [Info] Number of data points in the train set: 50479, number of used features: 90
[LightGBM] [Info] Start training from score 1959077.578795
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008880 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory 

[I 2025-06-04 01:02:58,171] Trial 1 finished with value: 778529.5193881042 and parameters: {'num_leaves': 45, 'learning_rate': 0.05613946202155921, 'n_estimators': 541}. Best is trial 0 with value: 731489.4631806696.

 64%|███████████████████████████▌               | 32/50 [11:31<05:18, 17.69s/it]

...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007194 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1069
[LightGBM] [Info] Number of data points in the train set: 50479, number of used features: 90
[LightGBM] [Info] Start training from score 1966133.280770
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008437 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1066
[LightGBM] [Info] Number of data points in the train set: 50479, number of used features: 90
[LightGBM] [Info] Start training from score 1959077.578795
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007779 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory 

[I 2025-06-04 01:10:42,042] Trial 28 finished with value: 729935.657262458 and parameters: {'num_leaves': 20, 'learning_rate': 0.2199761426536558, 'n_estimators': 937}. Best is trial 28 with value: 729935.657262458.

59it [19:15, 15.15s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009022 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1069
[LightGBM] [Info] Number of data points in the train set: 50479, number of used features: 90
[LightGBM] [Info] Start training from score 1966133.280770
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008725 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1066
[LightGBM] [Info] Number of data points in the train set: 50479, number of used features: 90
[LightGBM] [Info] Start training from score 1959077.578795
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory 

[I 2025-06-04 01:11:04,727] Trial 29 finished with value: 756440.7668716877 and parameters: {'num_leaves': 44, 'learning_rate': 0.21861089125396205, 'n_estimators': 939}. Best is trial 28 with value: 729935.657262458.

Tuning models:  50%|██████████████▌              | 2/4 [19:33<19:09, 574.98s/it][I 2025-06-04 01:11:04,731] A new study created in memory with name: no-name-648f0f2c-fa53-411f-8c23-413879a7b4d0
[I 2025-06-04 01:11:33,385] Trial 0 finished with value: 780945.9285754373 and parameters: {'depth': 8, 'learning_rate': 0.048272840390416745, 'iterations': 728}. Best is trial 0 with value: 780945.9285754373.

61it [20:07, 20.78s/it][I 2025-06-04 01:12:08,445] Trial 1 finished with value: 690130.5048388718 and parameters: {'depth': 8, 'learning_rate': 0.13124769075930845, 'iterations': 929}. Best is trial 1 with value: 690130.5048388718.

62it [20:42, 25.07s/it][I 2025-06-04 01:12:22,873] Trial 2 finished with value: 701852.1314063268 and parameters: {'depth': 7, 'learning_rate':

{'xgb': {'max_depth': 7,
  'learning_rate': 0.14005821753397463,
  'n_estimators': 998},
 'lgb': {'num_leaves': 20,
  'learning_rate': 0.2199761426536558,
  'n_estimators': 937},
 'cat': {'depth': 9, 'learning_rate': 0.18537299613880068, 'iterations': 979},
 'rf': {'max_depth': 20, 'n_estimators': 783}}

In [18]:
best_hyperparams

{'xgb': {'max_depth': 7,
  'learning_rate': 0.14005821753397463,
  'n_estimators': 998},
 'lgb': {'num_leaves': 20,
  'learning_rate': 0.2199761426536558,
  'n_estimators': 937},
 'cat': {'depth': 9, 'learning_rate': 0.18537299613880068, 'iterations': 979},
 'rf': {'max_depth': 20, 'n_estimators': 783}}